In [1]:
import os
import re
import math
import numpy as np
import pandas as pd
from scipy import stats
from typing import Union
import matplotlib.pyplot as plt
from matplotlib.figure import figaspect
import anndata as ad
import scanpy as sc
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from tqdm import tqdm 

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Load data

In [3]:
# read in protein data
data_dir = './../../../neighborhood/CRC_related/'
results_dir = './results_crc'
protein = pd.read_csv(os.path.join(data_dir, 'crc_codex/CRC_clusters_neighborhoods_markersV2.csv')) # ~258,000 codex cells

In [4]:
macro_list = ['CD68+CD163+ macrophages']

In [5]:
protein_macro = protein[protein['ClusterName'].isin(macro_list)]

In [6]:
rna_adata = ad.read(os.path.join(data_dir, './hacohen_scrna/data/rna_processed_macro_0311.h5'))

In [7]:
rna_adata

AnnData object with n_obs × n_vars = 18976 × 43113
    var: 'gene_ids', 'feature_types', 'genome'

In [8]:
sc.pp.normalize_total(rna_adata)
sc.pp.log1p(rna_adata)
sc.pp.highly_variable_genes(rna_adata, n_top_genes=2000)

rna_adata_2k = rna_adata[:, rna_adata.var.highly_variable].copy()
rna_adata_2k_df = pd.DataFrame(rna_adata_2k.X.toarray(), columns=rna_adata_2k.var_names)

In [9]:
matching = pd.read_csv('./../../multiomics_spatial/multiomics_fusion/results/crc_codex_rna_macro_163_0318.csv', index_col=False)
matching.shape

(31587, 3)

In [10]:
np.max(matching['mod2_indx'])

39595

In [11]:
np.max(matching['mod1_indx'])

18975

In [12]:
protein_new = protein_macro

In [13]:
protein_new = protein_new.reset_index()

In [14]:
protein_new['mod2_indx'] = protein_new.index

In [15]:
protein_rna = protein_new.reset_index() \
                .merge(matching, on='mod2_indx', how='left') \
                .merge(rna_adata_2k_df.reset_index().rename(columns={'index': 'mod1_indx'}), on='mod1_indx', how='left')

In [16]:
protein_rna

,level_0,index,Unnamed: 0.1,Unnamed: 0,CellID,ClusterID,EventID,File Name,Region,TMA_AB,...,RP13-228J13.1,AC006157.4,MT-RNR1,MT-TV,MT-RNR2,MT-TL1,MT-TQ,MT-TS1,MT-TP,CH507-513H4.1
0,0,118271,118271,118271,118271,10658,8,reg001_A,reg001,A,...,0.0,0.0,1.865276,0.000000,4.111446,0.0,0.0,0.0,0.0,0.0
1,1,118272,118272,118272,118272,10658,10,reg001_A,reg001,A,...,0.0,0.0,5.166441,0.000000,5.767593,0.0,0.0,0.0,0.0,0.0
2,2,118273,118273,118273,118273,10658,13,reg001_A,reg001,A,...,0.0,0.0,3.129398,0.000000,4.988626,0.0,0.0,0.0,0.0,0.0
3,3,118274,118274,118274,118274,10658,16,reg001_A,reg001,A,...,0.0,0.0,4.636842,0.000000,5.207959,0.0,0.0,0.0,0.0,0.0
4,4,118275,118275,118275,118275,10658,17,reg001_A,reg001,A,...,0.0,0.0,0.000000,0.000000,2.668557,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39591,39591,157862,157862,157862,157862,10658,258373,reg070_B,reg070,B,...,0.0,0.0,3.328004,0.000000,2.670094,0.0,0.0,0.0,0.0,0.0
39592,39592,157863,157863,157863,157863,10658,258375,reg070_B,reg070,B,...,0.0,0.0,2.200291,0.000000,3.589954,0.0,0.0,0.0,0.0,0.0
39593,39593,157864,157864,157864,157864,10658,258378,reg070_B,reg070,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39594,39594,157865,157865,157865,157865,10658,258381,reg070_B,reg070,B,...,0.0,0.0,5.796338,0.000000,6.135330,0.0,0.0,0.0,0.0,0.0


# Merge with hotspot data

In [17]:
hotspot = pd.read_csv('cells_in_hotspots.csv')
coldspot = pd.read_csv('cells_in_coldspots.csv')

In [27]:
# 1 is CLR, 2 is DII
hotspot['groups']

0        1
1        1
2        1
3        1
4        1
        ..
54122    2
54123    2
54124    2
54125    2
54126    2
Name: groups, Length: 54127, dtype: int64

In [26]:
list(hotspot.columns)

['Unnamed: 0.1',
 'Unnamed: 0',
 'CellID',
 'ClusterID',
 'EventID',
 'File Name',
 'Region',
 'TMA_AB',
 'TMA_12',
 'Index in File',
 'groups',
 'patients',
 'spots',
 'CD44 - stroma:Cyc_2_ch_2',
 'FOXP3 - regulatory T cells:Cyc_2_ch_3',
 'CD8 - cytotoxic T cells:Cyc_3_ch_2',
 'p53 - tumor suppressor:Cyc_3_ch_3',
 'GATA3 - Th2 helper T cells:Cyc_3_ch_4',
 'CD45 - hematopoietic cells:Cyc_4_ch_2',
 'T-bet - Th1 cells:Cyc_4_ch_3',
 'beta-catenin - Wnt signaling:Cyc_4_ch_4',
 'HLA-DR - MHC-II:Cyc_5_ch_2',
 'PD-L1 - checkpoint:Cyc_5_ch_3',
 'Ki67 - proliferation:Cyc_5_ch_4',
 'CD45RA - naive T cells:Cyc_6_ch_2',
 'CD4 - T helper cells:Cyc_6_ch_3',
 'CD21 - DCs:Cyc_6_ch_4',
 'MUC-1 - epithelia:Cyc_7_ch_2',
 'CD30 - costimulator:Cyc_7_ch_3',
 'CD2 - T cells:Cyc_7_ch_4',
 'Vimentin - cytoplasm:Cyc_8_ch_2',
 'CD20 - B cells:Cyc_8_ch_3',
 'LAG-3 - checkpoint:Cyc_8_ch_4',
 'Na-K-ATPase - membranes:Cyc_9_ch_2',
 'CD5 - T cells:Cyc_9_ch_3',
 'IDO-1 - metabolism:Cyc_9_ch_4',
 'Cytokeratin - epithel

In [18]:
protein_rna['hotspot'] = protein_rna['CellID'].isin(hotspot['CellID'])

In [19]:
protein_rna['coldspot'] = protein_rna['CellID'].isin(coldspot['CellID'])

In [20]:
np.sum(protein_rna['hotspot'])

9706

In [21]:
np.sum(protein_rna['coldspot'])

667

In [22]:
protein_rna.to_csv('protein_rna_matched_CRC_hotspot_macro_163.csv')